In [1]:
import torch 
import pickle
import io
import pandas as pd
import nltk

from modules.models import EncoderDecoder, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset
from transformers import BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from rouge import Rouge

In [5]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nl

True

In [2]:
checkpoint = torch.load('./checkpoints/encoder-decoder-lstm/sum-model/exp-2/checkpoint-epoch-11', map_location=torch.device('cpu'))

In [3]:
encoder_config = checkpoint['encoder_config']
decoder_config = checkpoint['decoder_config']

model = EncoderDecoder(encoder_config, decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [24]:
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

In [5]:
test = tokenizer(['Gestern Abend ging er in den Park um Fussball zu spielen.'], padding=True, truncation=True, return_tensors='pt')
reconstruction = model.reconstruct(test, n_beams=5)
tokenizer.decode(reconstruction[0][0], skip_special_tokens=True)

'Gesterns Abend ging er in den Park umhameneca zu spielen.'

In [6]:
test = tokenizer(['Gestern Abend ging er in den Park um Fussball zu spielen.', 'Dort hat er dann ein schönes Tor geschossen'], padding=True, truncation=True, return_tensors='pt')
summary = model.summarize_sequences(test)
tokenizer.decode(summary[0][0], skip_special_tokens=True)

'" Dass es hatte er den Startzeit in dann rein Leben liegen.'

In [33]:
## loading MLSUM datset
dataset = load_dataset("mlsum", "de", split="test")

## Wikilingua dataset
# dataset = load_dataset("wiki_lingua", "german", split="test")

Downloading: 4.48kB [00:00, 1.93MB/s]                   
Downloading: 12.5kB [00:00, 4.32MB/s]                   


Downloading: 100%|██████████| 17.7M/17.7M [00:00<00:00, 26.8MB/s]
Downloading: 100%|██████████| 311M/311M [00:08<00:00, 38.1MB/s]
Downloading: 100%|██████████| 17.8M/17.8M [00:01<00:00, 16.3MB/s]
100%|██████████| 3/3 [00:07<00:00,  2.54s/it]
                                           

Dataset mlsum downloaded and prepared to /Users/d059221/.cache/huggingface/datasets/mlsum/de/1.0.0/77f23eb185781f439927ac2569ab1da1083195d8b2dab2b2f6bbe52feb600688. Subsequent calls will reuse this data.


In [ ]:
dataset['summary'][:2]

In [ ]:
dataset['text'][:2]

In [8]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

with open('./data/train_set-1.pt', mode='rb') as f:
  train_set = CPU_Unpickler(f).load()

with open('./data/test_set-1.pt', mode='rb') as f:
  test_set = CPU_Unpickler(f).load()

AttributeError: Can't get attribute 'SummaryDataset' on <module '__main__'>

In [43]:
test = None
for i, (x1, x2, y) in enumerate(DataLoader(test_set, batch_size=None)):
    if i == 2:
        test = x1
        break
    

In [44]:
test['input_ids'][0,:].shape

torch.Size([28])

In [46]:
print(tokenizer.decode(test['input_ids'][0,:], skip_special_tokens=True))
reconstruction = model.reconstruct({'input_ids': test['input_ids'][0:1,:]}, n_beams=5)
print(tokenizer.decode(reconstruction[0][0], skip_special_tokens=True))

Das teilte der 31 - Jährige am Dienstagabend auf Instagram mit.
Das teilte der 31 - Jährige am Dienstagabend auf Instagram mit.


In [48]:
print(tokenizer.batch_decode(test['input_ids'][:,:], skip_special_tokens=True))

['Das teilte der 31 - Jährige am Dienstagabend auf Instagram mit.', '", hieß es in dem Post weiter.', '" Im Spiel ist der einfach vor eine Wand gelaufen, schmeißt sich auf den Boden und fängt an zu heulen.', 'Wenn ich den schubse, dann bleibt der stehen.']


In [47]:
summary = model.summarize_sequences(test)
tokenizer.decode(summary[0][0], skip_special_tokens=True)

'" Das ist die vor und der Spiel stecken, in den dann!'

## Run example summarization

In [16]:
df = pd.read_csv('/Users/d059221/Workspace/Git/master-thesis/data/export-articles-de-2021-09-08T14:45:27.515899+00:00')
df = df[['CONTENT']].dropna()

Split example article into sentences

In [17]:
df_test = df.iloc[:15000]
df_test['sentences'] = df_test['CONTENT'].apply(
    lambda x: nltk.sent_tokenize(
        x, language='german'))
df_test['sentences'] = df_test['sentences'].apply(
    lambda x: [
        s for s in x if len(
            s.split(' ')) > 4 and len(
            s.split(' ')) < 32])

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


build clusters for the sentences

In [116]:
sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
i_example = 0

example_sentences = df_test['sentences'].iloc[i_example]
#example_sentences = [
#    'Die Grünen ziehen mit Annalena Baerbock als Kanzlerkandidatin in die Bundestagswahl.',
#    'Das teilte Co-Parteichef Robert Habeck am Montag bei einem gemeinsamen Auftritt mit Baerbock in Berlin mit.', 
#    '„Wir haben eine klare Idee einer Kanzlerschaft für Deutschland“, sagte die 40-Jährige.', 
#    'Die Entscheidung muss noch auf einem Parteitag vom 11. bis 13. Juni bestätigt werden, die Zustimmung gilt aber als sicher.', 
#    'Noch vor drei Jahren war Annalena Baerbock in der Öffentlichkeit weithin unbekannt, nun schreibt sie für die Grünen Geschichte.', 
#    'Die 40-Jährige ist die erste Kanzlerkandidatin ihrer Partei.', 
#    'Mit der Realpolitikerin will die einstige Öko- und Friedenspartei bei der Bundestagswahl im September zur stärksten politischen Kraft aufsteigen.'
#]
sent_emb = sbert.encode(example_sentences)

num_sentences = len(example_sentences)
num_clusters = int(num_sentences * 0.5)
num_clusters = 1 if num_clusters == 0 else num_clusters
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sent_emb)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(example_sentences[sentence_id])

In [118]:
summary = ''
summ_sentences = list()
for cluster in clustered_sentences:
    tok = tokenizer(cluster, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")
    tok_tensor = torch.tensor(tok['input_ids'].numpy()[:,1:])
    summ_sentence = model.summarize_sequences({'input_ids': tok_tensor})
    summ_sentence = tokenizer.decode(summ_sentence[0][0], skip_special_tokens=True)
    print(summ_sentence)
    summ_sentences.append(summ_sentence)

summary = ' '.join(summ_sentences)

Ich glaube, dass es Biden - Vorgänger Donald Trump angreifen wird. Die USA haben am Montag in der Nacht zum Donnerstag begonnen. Quelle : ntv. de, mba / dpa
Das russische Außenministerium des Vereinigten Königreiches und die USA haben in den vergangenen Tagen eine Reihe von Bedrohungen in den USA zu tun. Quelle : ntv. de, mba / dpa
Er fügte hinzu : Wir wollen uns aufhören, das zu tun, was wir uns in den vergangenen Jahren machen können. Die Regierung in Washington hatte. Quelle : ntv. de, jpe / dpa
Ein hochrangiger Militärputsch sei unfair, das ist das Ergebnis. Quelle : ntv. de, mba / dpa
Am Donnerstag sagte der US - Präsident, nach seinem Amtsantritt im Januar zu Beginn der Pandemie werde.
Er sagte Biden auf Twitter, es gehe darum gehe, dass wir das Land in dieser Sache nicht. "
Unter den Unterstützer der Regierung in Washington seien auch mehrere Mitarbeiter des russischen Inlands, das US - Außenministerium.
Daran hat auch der Regierungswechsel in Washington im November 2020 begonne

Get the rouge scores

In [120]:
example_article = df_test['CONTENT'].iloc[i_example]
rouge = Rouge()
rouge.get_scores(summary, example_article)

[{'rouge-1': {'r': 0.1362126245847176,
   'p': 0.5256410256410257,
   'f': 0.21635883578114887},
  'rouge-2': {'r': 0.020964360587002098,
   'p': 0.10869565217391304,
   'f': 0.03514938217487612},
  'rouge-l': {'r': 0.1362126245847176,
   'p': 0.5256410256410257,
   'f': 0.21635883578114887}}]

Get the METEOR score

In [56]:
nltk.translate.meteor_score.meteor_score(example_article, summary)

0.03246753246753247

In [107]:
df_test['CONTENT'].iloc[1]

'Die Grünen ziehen mit Annalena Baerbock als Kanzlerkandidatin in die Bundestagswahl. Das teilte Co-Parteichef Robert Habeck am Montag bei einem gemeinsamen Auftritt mit Baerbock in Berlin mit. „Wir haben eine klare Idee einer Kanzlerschaft für Deutschland“, sagte die 40-Jährige.Die Entscheidung muss noch auf einem Parteitag vom 11. bis 13. Juni bestätigt werden, die Zustimmung gilt aber als sicher.Noch vor drei Jahren war Annalena Baerbock in der Öffentlichkeit weithin unbekannt, nun schreibt sie für die Grünen Geschichte. Die 40-Jährige ist die erste Kanzlerkandidatin ihrer Partei. Mit der Realpolitikerin will die einstige Öko- und Friedenspartei bei der Bundestagswahl im September zur stärksten politischen Kraft aufsteigen.Mehr in Kürze'

## Use sub-doc approach

In [63]:
checkpoint = torch.load('./checkpoints/encoder-decoder-lstm/sum-model/exp-4/checkpoint-epoch-8', map_location=torch.device('cpu'))

encoder_config = checkpoint['encoder_config']
decoder_config = checkpoint['decoder_config']

model = EncoderDecoder(ModelConfig(encoder_config), ModelConfig(decoder_config))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
print('')

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

with open('./data/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = CPU_Unpickler(f).load()

with open('./data/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = CPU_Unpickler(f).load()

In [8]:
test_set.dataset.df['split_doc'].str.len().sum()

138208

In [121]:
i_example = 0
sample = test_set.dataset.df.iloc[i_example]['enc_tokens']
summary = model.summarize_sequences(sample)
summary = tokenizer.decode(summary[0][0], skip_special_tokens=True)
print(summary)

Am Donnerstag verkündete Biden der USA sich an die US - Regierung und Biden. Biden sagte am Montag, er wolle nicht mit dem russischen Präsidenten Wladimir Putin sprechen. Die US - Außenminister Antony Blinken und die US - Außenminister Antony Blinken und die US - Außenminister Antony Blinken und US - Außenminister Antony Blinken wollen nach Angaben von US - Außenminister Antony Blinken in einem Telefonat mit dem russischen Präsidenten Wladimir Putin auf die Frage nach möglichen Sanktionen gegen den Kreml - Kritiker Alexej Nawalny reagiert werden. Ich bin sehr besorgt, sagte Biden


In [122]:
example_article = df_test['CONTENT'].iloc[i_example]
rouge.get_scores(summary, example_article)

[{'rouge-1': {'r': 0.10631229235880399,
   'p': 0.6037735849056604,
   'f': 0.1807909579059338},
  'rouge-2': {'r': 0.018867924528301886,
   'p': 0.13846153846153847,
   'f': 0.03321032999244305},
  'rouge-l': {'r': 0.10299003322259136,
   'p': 0.5849056603773585,
   'f': 0.17514124039180953}}]

In [109]:
example_article

'Bei neuen gewaltsamen Auseinandersetzungen zwischen Palästinensern und israelischen Polizisten in Jerusalem sind am Samstagabend nach Angaben von Rettungskräften mehr als 50 Menschen verletzt worden. „Wir zählen bislang 53 Verletzte“, sagte ein Sprecher des palästinensischen Roten Halbmonds der Nachrichtenagentur AFP.\xa0\nZu den neuen Zusammenstößen kam es in der Altstadt von Jerusalem und im Ost-Jerusalemer Stadtteil Scheich Dscharrah. Dort gingen erneut Palästinenser auf die Straße, um gegen drohende Zwangsräumungen zu protestieren, und warfen mit Steinen in Richtung israelischer Sicherheitskräfte. Diese setzten Wasserwerfer ein und versuchten, die Demonstranten mit übelriechendem Wasser zu vertreiben, wie ein AFP-Korrespondent berichtete.\nJugendliche zündeten laut Nachrichtenagentur Reuters zudem Feuer an und zerstörten Barrikaden, die den Zugang zur Altstadt versperren sollten.\nBereits am Abend zuvor hatte es Zusammenstöße auf dem Tempelberg und in anderen Teilen Ost-Jerusalems